# Graphics Processing Units (GPUs)

- Originally designed to perform computationally intensive graphics calculations
- GPUs as we know them today are largely derived from the NVIDIA GeForce 256 card, which was the first consumer level GPU
- Quickly, scientists and engineers recognized the similarities in many of their problems and those that the GPU was designed to be very good at. This led to what we now refer to as General Purpose GPU (GPGPU) programming.
    - Any problem which requires the application of similar operations to a lot of data can benefit from the architecture of GPUs
- Though GPU hardware is capable of accelerating computation in many fields other than just graphics, to take full advantage of GPUs a software platform that allows GPU programs to be written similarly to CPU programs. The first to solve this problem was NVIDIA with CUDA paradigm.
    - Soon after a vendor-agnostic language was created (OpenCL)
- CUDA and OpenCL turned GPUs into general purpose computing devices that can be leveraged alongside CPUs to perform complex calculations

# GPU vs. CPU

- GPUs are "external" devices that must interface with a CPU 
- GPUs cores are usually around 2 times slower than CPUs (2-2.5 GHz compared to 4-6 GHz)
- GPUs have **a lot** more cores than CPUs
    - For example, an AMD Threadripper 3990X has 64 cores (which is about as much as you will find) while an NVIDIA RTX 3090 has more than 10000 CUDA cores. 
- This means that processes that are algorithmically single-threaded or not compute intensive will generally be faster on CPUs than GPUs and vice-versa.

| ![CPU vs. GPU](./figures/gpu-devotes-more-transistors-to-data-processing.png) |
|:--:|
| Figure Credit - Figure 1 (https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html) |


# GPU Terminology

- **Host** - the CPU that the GPU is connected to
- **Device** - the GPU
- **GPU Core** - processing unit of GPU
- **GPU Thread** - execution context of a GPU core
- **Kernel** - function created to be executed on the GPU

# Model for Parallelism on GPUs

GPUs excel at what is commonly referred to as Single Instruction Multiple Data (SIMD) parallelism. This means that any time you write a loop to do computation, there is potential that it can be accelerated with a GPU. 

To best understand how GPUs are used, we can look at the CUDA programming paradigm. Though there are other ways to program GPUs (OpenACC, OpenCL), developing a basic understanding of CUDA makes using these other methods significantly easiser.

- The basic idea is to parallelize iteration blocks by replacing the iteration variable with a thread index variable.
    - So write the body of the loop as a separate function that is then executed on the GPU (**Kernel**)
- Threads are organized into (possible multidimensional) **blocks**
- Blocks are divided into **warps** where the number of threads in a warp is equal to the number of cores in a **streaming multiprocessor** (**SM**)
    - SMs are just collections of cores, where each core has an arithemetic logic unit (ALU) and a floating point unit (FPU)
- Warps are then executed on SMs
- As programmers, we are usually only concerned with threads and blocks. For this reason, CUDA provides several predifined variables to help determine which thread is running what. 
    - `gridDim` - grid dimensions (in blocks, e.g., a grid can be 540 x 480 blocks)
    - `blockDim` - block dimensions (in threads)
    - `blockIdx` - block position within a grid
    - `threadIdx` - thread position within a block
    - e.g. To process a 4K image (2160 x 3840 pixels), you may choose a block size of 32 x 32 (choosing block sizes that are multiples of 32 is good for performance as that is the number of threads in an SM). With that block size the grid size will be 68 x 120 (ceil(2160/32), ceil(3840/32)). Note this is a 2 dimensional grid. So to get the x index: `int i_x = blockIdx.x * blockDim.x + threadIdx.x` and to get the y index: `int i_y = blockIdx.y * blockDim.y + threadIdx.y`.

```C++
#include <stdio.h>
#define N 64
#define TPB 32

// any function the the __device__ qualifer is compiled to be
// called from GPU and executed on GPU
__device__ float scale(int i, int n) {
    return ((float) i)/(n-1);
}

__device__ float distance(float x1, x2) {
    return sqrt((x2 - x1) * (x2 - x1));
}

// functions with the __global__ qualifier is compiled to be
// called from CPU and executed on GPU
__global__ void distanceKernel(float *d_out, float ref, int len) {
    const int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < len) {
        const float x = scale(i, len);
        d_out[i] = distance(x, ref);
    }
    // it is pretty simple to see how the const int i line
    // is replacing a for (int i=0; i<N; ++i) loop
    // this is often referred to as going from loops to grids
}

// you can also stack qualifiers so
// __host__ __device__ functions will have a CPU
// and CPU version compiled and the correct one will be ran 
// depending on where they are called on.

int main() {
    const float ref = 0.5f;
    
    // pointer for array of floats
    float *d_out = 0;
    
    // allocate device memory for output array
    cudaMalloc(&d_out, N*sizeof(float));
    
    // launch kernel (1d)
    distanceKernel<<<N/TPB, TPB>>>(d_out, ref, N);
    
    cudaFree(d_out);
    return 0;
}
    
```

# GPGPU Progamming Considerations

- GPU code execution is asyncronous by nature. You can never when a particular index will be calculated so there can be no data dependencies between indices.
```C++
// so this is not valid
d[i] = (d[i-1] + d[i-2]) / 2
```
- GPUs have their own memory, often referred to as VRAM. When writing code in C, C++, or FORTRAN, you must take care to manage both CPU and GPU memory to get the maximum performance regardless of what paradigm you are using. You can generally follow this process:
    - Allocate memory on device (`cudaMalloc`) and host (`malloc`)
    - *Prepare data on host*
    - Transfer data from host to device (`cudaMemcpy`, direction: `cudaMemcpyHostToDevice`)
    - *Process data on device*
    - Transfer results back to host (`cudaMemcpy`, direction: `cudaMemcpyDeviceToHost`)
    - Free memory on device (`cudaFree`) and host (`free`) 
    - With OpenACC and OpenCL, you can are able to avoid many of this as they manage the device memory for you; however, in many cases you can end up with slower code unless you add `data` directives as they manage it conservatively. 
- For data that is required for many calculations, it is better to persist it on the GPU if possible. Transfer of data from host to device is very slow and can easily overshadow any speedup benefits you get if not performed intelligently.

| ![gpu memory bandwidth](./figures/memory_bandwidth.jpg) |
|:--:|
| Figure Credit: https://github.com/maxim-belkin/how_to_gpu_in_python |


# OpenACC

- OpenACC is an attractive alternative to CUDA as it enables directives based code acceleration similar to OpenMP
- The NVIDIA HPC SDK, formerly the PGI compiler suite, allows users to compile code for GPUs that is **very** similar to CPU specific code. 
    - On Henry2, I have had success using the `PrgEnv-pgi/19.4` module to compile code for GPUs
- Rather than separating the body of a loop into a separate function that can only run on the device, converting a loop to a grid for a GPU can be as simple as adding the `#pragma acc kernels` directive before the loop and the compiler will do the rest. 
- Additionally, by adding the `-ta=multicore` compiler flag you can compile the same code for a CPU and it will run similarly to as if you had used OpenMP.
    - This means that you can write your code once, but compile a version that can run on a GPU and a version that can run without a GPU, both of which will be parallelized.
- Further, rather than being tied to NVIDIA Hardware (as you are with CUDA), you can use the `-ta=radeon` flag to compile code that can be executed on AMD GPUs
    - NOTE: I have not tested this.
- The [OpenACC Programming and Best Practices Guide](https://www.openacc.org/sites/default/files/inline-files/openacc-guide.pdf) is a great place to start learning about OpenACC.
- The [OpenACC Specifications](https://www.openacc.org/specification) are where you want to look for detailed information about all the functionality it provides. 
- You can also find two OpenACC examples in the `examples/c++/openacc` folder, both of which include submission files that can be used to run the programs on Henry2.